In [33]:
using LinearAlgebra
using TensorKit
using KrylovKit
using JSON
using HDF5, JLD
cd("D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
#push!(LOAD_PATH, "D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
include("kagome_load_tensor.jl")
include("kagome_CTMRG.jl")
include("kagome_model.jl")
include("kagome_IPESS.jl")
include("mps_algorithms\\ITEBD_algorithms.jl")
include("mps_algorithms\\TransfOp_decomposition.jl")
include("mps_algorithms\\PUMPS_algorithms.jl")



D=8;
chi=60;
W=20;
N=20;
kset=0:4;
EH_n=3;#number of entanglement spectrum per k point
Dtrun_method="svds";
Dtrun_init=400;
Dtrun_max=400;


J1=0.80902;
J2=0;
J3=0;
Jchi=0;
Jtrip=0.5878;

parameters=Dict([("J1", J1), ("J2", J2), ("J3", J3), ("Jchi", Jchi), ("Jtrip", Jtrip)]);



Dict{String, Real} with 5 entries:
  "Jchi"  => 0
  "J1"    => 0.80902
  "Jtrip" => 0.5878
  "J3"    => 0
  "J2"    => 0

In [34]:

    multi_threads=true;if Threads.nthreads()==1; multi_threads=false; end
    println("number of threads: "*string(Threads.nthreads()));flush(stdout);
    
    CTM_conv_tol=1e-6;
    trun_tol=1e-8;
    group_size=Int(round((10^8)/(chi*chi*W*W*D)));
    
    mpo_type="OO";#"O_O" or "OO", in my test "OO" is faster for large bond dimension
    
    pow=Int((N-2)/2);
    

    
    A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb=construct_tensor(D);
    
    filenm="LS_D_"*string(D)*"_chi_40.json"
    json_dict=read_json_state(filenm);
    
    bond_tensor,triangle_tensor=construct_su2_PG_IPESS(json_dict,A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb);
    
    PEPS_tensor=bond_tensor;
    @tensor PEPS_tensor[:] := bond_tensor[-1,1,-5]*bond_tensor[4,3,-6]*bond_tensor[-4,2,-7]*triangle_tensor[1,3,2]*triangle_tensor[4,-2,-3];
    A_unfused=PEPS_tensor;
    
    U_phy=unitary(fuse(space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7)), space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7));
    @tensor A_fused[:] :=PEPS_tensor[-1,-2,-3,-4,1,2,3]*U_phy[-5,1,2,3];

number of threads: 1


In [35]:
CTM=[];
U_L=[];
U_D=[];
U_R=[];
U_U=[];


println("No CTM found from saved data, now do CTMRG");flush(stdout);


init=Dict([("CTM", []), ("init_type", "PBC")]);
conv_check="singular_value";
@time CTM, AA_fused, U_L,U_D,U_R,U_U=CTMRG(A_fused,chi,conv_check,CTM_conv_tol,init);


No CTM found from saved data, now do CTMRG


"initialize CTM"

"start CTM iterations:"

  0.279436 seconds (783.65 k allocations: 77.595 MiB, 12.35% gc time, 53.41% compilation time)
CTMRG iteration: 1, CTMRG err: 6.386909242189306
CTMRG iteration: 2, CTMRG err: 0.5162630418442805
CTMRG iteration: 3, CTMRG err: 0.42386112908678647
CTMRG iteration: 4, CTMRG err: 0.2009231399861225
CTMRG iteration: 5, CTMRG err: 0.1488093148145859
CTMRG iteration: 6, CTMRG err: 0.08626209897556224
CTMRG iteration: 7, CTMRG err: 0.07839382418077667
CTMRG iteration: 8, CTMRG err: 0.04155417608905354
CTMRG iteration: 9, CTMRG err: 0.0459018326933857
CTMRG iteration: 10, CTMRG err: 0.021433388245652364
CTMRG iteration: 11, CTMRG err: 0.028505354528318365
CTMRG iteration: 12, CTMRG err: 0.01173182211588852
CTMRG iteration: 13, CTMRG err: 0.01841340155206256
CTMRG iteration: 14, CTMRG err: 0.0069133839020670594
CTMRG iteration: 15, CTMRG err: 0.012254667375128155
CTMRG iteration: 16, CTMRG err: 0.004436920839694985
CTMRG iteration: 17, CTMRG err: 0.00835692651047851
CTMRG iteration: 18, CTMRG err

In [36]:


@time E_up, E_down=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, U_L,U_D,U_R,U_U, CTM, "E_triangle");
@time E_up_12, E_up_31, E_up_23, E_down_12, E_down_31, E_down_23=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, U_L,U_D,U_R,U_U, CTM, "E_bond");
println((E_up+E_down)/3);flush(stdout);


  8.154295 seconds (6.02 M allocations: 2.186 GiB, 52.09% gc time, 15.51% compilation time)
  5.926864 seconds (4.18 M allocations: 1.984 GiB, 45.56% gc time, 1.04% compilation time)
-0.40397758467142536 - 1.4395520461967011e-10im


In [ ]:
O1O2


In [37]:
function build_double_layer_extra_leg(A,operator)
    #su2 operator has three legs, such as svd decomposition of Heisenberg interaction 
    #first two indices of operator are physical indices
    A=permute(A,(1,2,),(3,4,5));
    U_L=unitary(fuse(space(A, 1)' ⊗ space(A, 1)), space(A, 1)' ⊗ space(A, 1));
    U_D=unitary(fuse(space(A, 2)' ⊗ space(A, 2)), space(A, 2)' ⊗ space(A, 2));
    U_R=inv(U_L);
    U_U=inv(U_D);
    # display(space(U_L))
    # display(space(U_D))
    # display(space(U_R))
    # display(space(U_D))

    uM,sM,vM=tsvd(A);
    uM=uM*sM

    uM=permute(uM,(1,2,3,),())
    V=space(vM,1);
    U=unitary(fuse(V' ⊗ V), V' ⊗ V);
    @tensor double_LD[:]:=uM'[-1,-2,1]*U'[1,-3,-4];
    @tensor double_LD[:]:=double_LD[-1,-3,1,-5]*uM[-2,-4,1];
    
    vM=permute(vM,(1,2,3,4,),());

        @tensor double_RU[:]:=U[-1,-2,1]*vM[1,-3,-4,-5];
        @tensor double_RU[:]:=vM'[3,-2,-4,1]*operator[2,1,-6]*double_RU[-1,3,-3,-5,2];
 
    #display(space(double_RU))

    double_LD=permute(double_LD,(1,2,),(3,4,5,));
    double_LD=U_L*double_LD;
    double_LD=permute(double_LD,(2,3,),(1,4,));
    double_LD=U_D*double_LD;
    double_LD=permute(double_LD,(2,1,),(3,));
    #display(space(double_LD))
    double_RU=permute(double_RU,(1,4,5,6,),(2,3,));
    double_RU=double_RU*U_R;
    double_RU=permute(double_RU,(1,5,4,),(2,3,));
    double_RU=double_RU*U_U;
    double_LD=permute(double_LD,(1,2,),(3,));
    double_RU=permute(double_RU,(1,),(2,4,3,));
    AA_fused=double_LD*double_RU;
    AA_fused=permute(AA_fused,(1,2,3,4,5,),());
    
    return AA_fused, U_L,U_D,U_R,U_U
end



build_double_layer_extra_leg (generic function with 1 method)

In [ ]:
function single_spin_operator(U_phy,posit1,posit2)

    # Heisenberg interaction
    Id=TensorMap(I(2),space(U_phy,4),space(U_phy,4));
    sx=[[0,1] [1,0]]/2; sy=[[0,1] [-1,0]]/2*im; sz=[[1,0] [0,-1]]/2;
    @tensor HSS[:]:=sx[-1,-3]*sx[-2,-4]+sy[-1,-3]*sy[-2,-4]+sz[-1,-3]*sz[-2,-4];
    HSS=TensorMap(HSS, space(U_phy,4)⊗space(U_phy,4) ← space(U_phy,4)⊗space(U_phy,4));
    HSS=permute(HSS,(1,3,),(2,4,));
    u,s,v=tsvd(HSS);
    H1=u*s;
    H2=permute(v,(2,3,),(1,));

    if posit1==1
        @tensor H1[:]:= U_phy'[4,5,6,-1]*H1[1,4,-3]*Id[2,5]*Id[3,6]*U_phy[-2,1,2,3];
    elseif posit1==2
        @tensor H1[:]:= U_phy'[4,5,6,-1]*Id[1,4]*H1[2,5,-3]*Id[3,6]*U_phy[-2,1,2,3];
    elseif posit1==3
        @tensor H1[:]:= U_phy'[4,5,6,-1]*Id[1,4]*Id[2,5]*H1[3,6,-3]*U_phy[-2,1,2,3];
    end

    if posit2==1
        @tensor H2[:]:= U_phy'[4,5,6,-1]*H2[1,4,-3]*Id[2,5]*Id[3,6]*U_phy[-2,1,2,3];
    elseif posit2==2
        @tensor H2[:]:= U_phy'[4,5,6,-1]*Id[1,4]*H2[2,5,-3]*Id[3,6]*U_phy[-2,1,2,3];
    elseif posit2==3
        @tensor H2[:]:= U_phy'[4,5,6,-1]*Id[1,4]*Id[2,5]*H2[3,6,-3]*U_phy[-2,1,2,3];
    end


    return H1,H2 
end


In [38]:
display(space(U_phy))

Rep[SU₂](1/2=>2, 3/2=>1) ← (Rep[SU₂](1/2=>1)' ⊗ Rep[SU₂](1/2=>1)' ⊗ Rep[SU₂](1/2=>1)')